# Imports

In [1]:
%load_ext autoreload
%autoreload 0

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, r2_score

In [4]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

# RandomForestRegressor

In [5]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestBunch(model, X_test, y_test, r2_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names)

db = RandomForestDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False,
                        shadow_trees=True)
db.run(8051)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating shap values...
Calculating importances...
Running Model Explainer on http://localhost:8051
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2019 11:46:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:16] "GET /assets/stylesheet.css?m=1570631370.0 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:16] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:16] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:16] "GET /assets/bootstrap.css?m=1570631370.0 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:17] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:17] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=15

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1287, in dispatch
    response.set_data(self.callback_map[output]['call

127.0.0.1 - - [08/Nov/2019 11:46:48] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [08/Nov/2019 11:46:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:46:49] "POST /_dash-update-component HTTP/1.1" 200 -


# RandomForestClassifier:

In [6]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)


explainer = RandomForestClassifierBunch(model, X_test, y_test, roc_auc_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               labels=['Not survived', 'Survived'])

db = RandomForestDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=True)
db.run(8052)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction probabilities...
Calculating importances...
Calculating shap interaction values...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2019 11:47:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /assets/stylesheet.css?m=1570631370.0 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /assets/bootstrap.css?m=1570631370.0 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:47:34] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=15

Generating shadow trees...
Generating shadow trees...
Generating shadow trees...


127.0.0.1 - - [08/Nov/2019 11:48:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:48:42] "POST /_dash-update-component HTTP/1.1" 200 -


# XGBClassifier:

In [5]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

explainer = TreeModelClassifierBunch(xgb_model, X_test, y_test, roc_auc_score, 
                                   cats=['Sex', 'Cabin', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction probabilities...
Calculating importances...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2019 11:52:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:46] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566138672 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1566138674 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1